In [8]:
import pandas as pd

In [9]:
movies = pd.read_csv('../datasets/movies.csv')
links = pd.read_csv('../datasets/links.csv')
ratings = pd.read_csv('../datasets/ratings.csv')

bechdel = pd.read_csv('../datasets/bechdel.csv')

In [10]:
movies.shape

(9742, 3)

In [11]:
links.shape

(9742, 3)

In [12]:
ratings.shape

(100836, 4)

In [13]:
ratings['userId'].nunique()

610

In [14]:
bechdel.shape

(9417, 10)

#### Merge Movielens: movies + links on 'movieId'

In [15]:
movielens = pd.merge(movies, links, left_on='movieId', right_on='movieId')

#### + ratings

In [16]:
movielens = pd.merge(movielens, ratings, left_on='movieId', right_on='movieId')

In [17]:
movielens.head(1)

,movieId,title,genres,imdbId,tmdbId,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,1,4.0,964982703


#### Drop redundant or unnecessary: 
'movieId', 'tmdbId', 'timestamp'

In [18]:
movielens.drop(columns=['movieId', 'tmdbId', 'timestamp'], inplace=True)
movielens.head(1)

,title,genres,imdbId,userId,rating
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,1,4.0


In [19]:
movielens.to_csv('../datasets/final.csv', index=False)

#### Rename Bechdel rating 

In [20]:
bechdel.head(1)

,Unnamed: 0,visible,date,title,rating,year,imdbid,submitterid,dubious,id
0,0,1,2022-01-13 14:47:49,"355, The",3,2022,8356942.0,19837,0.0,10195


In [21]:
bechdel.rename(columns={'rating': 'bechdel_score'}, inplace=True)
bechdel.head(1)

,Unnamed: 0,visible,date,title,bechdel_score,year,imdbid,submitterid,dubious,id
0,0,1,2022-01-13 14:47:49,"355, The",3,2022,8356942.0,19837,0.0,10195


#### Merge Movielens and Bechdel on 'imdbId' and 'imdbid' respectively

In [22]:
df = pd.merge(movielens, bechdel, left_on='imdbId', right_on='imdbid')

In [23]:
df.head(1)

,title_x,genres,imdbId,userId,rating,Unnamed: 0,visible,date,title_y,bechdel_score,year,imdbid,submitterid,dubious,id
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,1,4.0,6479,1,2008-07-28 15:55:31,Toy Story,1,1995,114709.0,1,NaN,87


#### Rename title_x

In [24]:
df.rename(columns={'title_x': 'title'}, inplace=True)
df.head(1)

,title,genres,imdbId,userId,rating,Unnamed: 0,visible,date,title_y,bechdel_score,year,imdbid,submitterid,dubious,id
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,1,4.0,6479,1,2008-07-28 15:55:31,Toy Story,1,1995,114709.0,1,NaN,87


#### Drop redundant or unnecessary: 
'Unnamed: 0', 'visible', 'date', 'title_y', 'year', 'imdbid', 'submitterid', 'id', 'dubious'

In [25]:
df.drop(columns=['Unnamed: 0', 'visible', 'date', 'title_y', 'year', 'imdbid', 'submitterid', 'id', 'dubious'], inplace=True)
df.head(1)

,title,genres,imdbId,userId,rating,bechdel_score
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,1,4.0,1


#### check for Null

In [26]:
df.isnull().sum()

title            0
genres           0
imdbId           0
userId           0
rating           0
bechdel_score    0
dtype: int64

In [27]:
df.shape

(82859, 6)

#### extract year from title as own column

In [28]:
pattern = '\((\d{4})\)'
df['year'] = df.title.str.extract(pattern)
df.head()

,title,genres,imdbId,userId,rating,bechdel_score,year
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,1,4.0,1,1995
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,5,4.0,1,1995
2,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,7,4.5,1,1995
3,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,15,2.5,1,1995
4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,17,4.5,1,1995


#### Save to csv

In [29]:
#df.to_csv('../datasets/all.csv', index=False)

#### 9742 movies 100836 ratings from 610 users, 9417 bechdel scores resulting in 

#### 4838 complete records and 82859 reviews